## Test notebook for Training Clasifier

In [21]:
import re
import pandas as pd
import numpy as np
import nltk
import sqlite3 as sq
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

In [2]:
def get_df():
    db = sq.connect('data/YourDatabaseName.db')
    df = pd.read_sql('SELECT * FROM messages_categories', con = db)
    
    return df

In [3]:
def partition_msg_cat(df):
    messages = df.message
    categories = df[df.columns[3:]]
    
    return messages,categories

In [4]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [36]:
def tokenize(text):
    text = re.sub(r'[^a-zA-Z0-9]'," ",text)
    
    tokens = word_tokenize(text.lower())
    
    stem_tk = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return stem_tk

In [37]:
df = get_df()
X,y =  partition_msg_cat(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)

In [38]:
vect = CountVectorizer(tokenizer = tokenize)
tfidf = TfidfTransformer(smooth_idf=False)
claf = RandomForestClassifier()

X_train_vec = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_vec)
claf.fit(X_train_tfidf,y_train)

RandomForestClassifier()

In [39]:
X_test_vec = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_vec)

In [40]:
y_preds = claf.predict(X_test_tfidf)

In [41]:
labels = np.unique(y_preds)
#confusion_matrix = confusion_matrix(y_test, y_preds, labels = labels)
accuracy = (y_test == y_preds).mean()

In [44]:
print(accuracy.mean())

0.9458094716748074


In [43]:
#conf_matrix = multilabel_confusion_matrix(y_test, y_preds, labels = labels)